## 1. 사용할 패키지 불러오기

To build an autoencoder-based anomaly detection model, we'll use Keras, an open-source deep learning library written in Python.

In [10]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

## 2. 데이터 불러오기

#### (1) Pandas의 read_excel 함수를 사용하면 xlsx 형식의 파일을 불러올 수 있다.

In [21]:
train_data = pd.read_excel('Train.xlsx', index_col=0)
valid_data = pd.read_excel('Test.xlsx', index_col=0)

#### (2) 데이터의 정보가 1번 Index부터 시작하므로 전처리를 해줘야 한다.

In [23]:
train_data.columns = train_data.iloc[0, :]
train_data = train_data.iloc[1:, ].reset_index(drop = True)

valid_data.columns = valid_data.iloc[0, :]
valid_data = valid_data.iloc[1:, 1:].reset_index(drop = True)

## 3. 데이터 정규화

Deep learning 모델은 X 변수들을 정규화해야 한다.  
이 때, Scikit-learn 패키지의 MinMaxScalar를 사용할 수 있다.

In [25]:
scaler = StandardScaler()
train_data_scaled = scaler.fit_transform(train_data)

## 4. Define AutoEncoder Model

Input Shape와 Encoding Dimension을 정해주면, Keras 패키지에 내장되어 있는 함수들로 Auto Encoder 모델을 만들 수 있다.

In [35]:
def autoencoder_model(encoding_dim):
    # Encoder
    input_layer = layers.Input(shape=52)
    encoded = layers.Dense(encoding_dim, activation='relu')(input_layer)
    
    # Decoder
    decoded = layers.Dense(52, activation='sigmoid')(encoded)
    
    # Autoencoder model
    autoencoder = models.Model(input_layer, decoded)

    autoencoder.compile(optimizer='adam', loss='mean_squared_error')

    
    
    return autoencoder

## 5. Hyper-parameter Setting

Scikit-learn 패키지의 GridSearchCV 함수를 사용하면, Hyper-parameter tuning을 수행할 수 있다.  
params (Dictionary)에 파라미터 목록을 추가하면 된다.

In [36]:
# Define the hyperparameters for grid search
param_grid = {'encoding_dim': [10, 20, 30, 40]}

## 6. Train the autoencoder using grid search

In [37]:
input_shape = train_data_scaled.shape[1]
autoencoder = tf.keras.wrappers.scikit_learn.KerasRegressor(
    build_fn=autoencoder_model, epochs=50, batch_size=32, verbose=0)
grid_search = GridSearchCV(autoencoder, param_grid=param_grid, cv=3)
grid_result = grid_search.fit(train_data_scaled, train_data_scaled)
best_encoding_dim = grid_result.best_params_['encoding_dim']

/tmp/ipykernel_26665/299675781.py:2: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  autoencoder = tf.keras.wrappers.scikit_learn.KerasRegressor(


In [38]:
# Fit final model with best hyperparameters
best_params = grid_result.best_params_
print('Best hyperparameters:', best_params)

Best hyperparameters: {'encoding_dim': 40}


In [40]:
autoencoder = autoencoder_model(best_encoding_dim)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')
autoencoder.fit(train_data_scaled, train_data_scaled, epochs=50, batch_size=32, verbose=0)

## 7. Prediction

In [41]:
# 검증 데이터셋 정규화 수행
valid_data_scaled = scaler.transform(valid_data)

# 학습 된 모델로 예측 값 생성
valid_data_pred = autoencoder.predict(valid_data_scaled)

# MSE를 활용한 Anomaly Score 생성
valid_data_mse = np.mean(np.power(valid_data_scaled - valid_data_pred, 2), axis=1)

In [42]:
valid_data['Process variables'] = valid_data_mse
valid_data['Process variables'] = np.where(valid_data['Process variables'] > valid_data['Process variables'].mean(), 1, 0)

In [43]:
valid_data.to_excel('AnomalyDetectionResults.xlsx')